In [12]:
%tensorflow_version 2.x
%load_ext tensorboard
import pandas as pd
import numpy as np
import random
import math
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import Dense, Dropout, LSTM,  Activation
from tensorflow.keras import optimizers
from google.colab import drive
from tensorflow import keras
import tensorflow as tf


from datetime import datetime
# import Data
drive.mount('/content/drive')

dataset0 = pd.read_csv('/content/drive/My Drive/Indycar/LSTM3/Data2013.csv')
dataset1 = pd.read_csv('/content/drive/My Drive/Indycar/LSTM3/Data2014.csv')
dataset2 = pd.read_csv('/content/drive/My Drive/Indycar/LSTM3/Data2015.csv')
dataset3 = pd.read_csv('/content/drive/My Drive/Indycar/LSTM3/Data2016.csv')
dataset4 = pd.read_csv('/content/drive/My Drive/Indycar/LSTM3/Data2017.csv')
dataset5 = pd.read_csv('/content/drive/My Drive/Indycar/LSTM3/Data2018.csv')

data0 = np.array(dataset0)[: ,1:]
data1 = np.array(dataset1)[: ,1:]
data2 = np.array(dataset2)[: ,1:]
data3 = np.array(dataset3)[: ,1:]
data4 = np.array(dataset4)[: ,1:]
data5 = np.array(dataset5)[: ,1:]

testset1 = pd.read_csv('/content/drive/My Drive/Indycar/LSTM3/Data2018.csv')
testset2 = pd.read_csv('/content/drive/My Drive/Indycar/LSTM3/Data2019.csv')
testdata1 = np.array(testset1)[: ,1:]
testdata2 = np.array(testset2)[: ,1:]

# ===============
'''
for i in range(33):
  data0[:, 2*i +1] = data0[:, 2*i +1]/2800
  data1[:, 2*i +1] = data1[:, 2*i +1]/2800
  data2[:, 2*i +1] = data2[:, 2*i +1]/2800
  data3[:, 2*i +1] = data3[:, 2*i +1]/3000
  data4[:, 2*i +1] = data4[:, 2*i +1]/3200
  data5[:, 2*i +1] = data5[:, 2*i +1]/3200

  testdata1[:, 2*i +1] = testdata1[:, 2*i +1]/3200
  testdata2[:, 2*i +1] = testdata2[:, 2*i +1]/3200
'''
for i in range(33):
  data0[:, 2*i +1] = data0[:, 2*i +1]/2800
  data1[:, 2*i +1] = data1[:, 2*i +1]/2800
  data2[:, 2*i +1] = data2[:, 2*i +1]/2800
  data3[:, 2*i +1] = data3[:, 2*i +1]/2800
  data4[:, 2*i +1] = data4[:, 2*i +1]/2800
  data5[:, 2*i +1] = data5[:, 2*i +1]/2800

  testdata1[:, 2*i +1] = testdata1[:, 2*i +1]/2800
  testdata2[:, 2*i +1] = testdata2[:, 2*i +1]/2800




def create_sample(dataset, t,pre, noise):
  predict = 340  # (15-1)*20 = 280, 280 + 60 =340
  
  step = 15
  predict = (step-1)*20+pre
  sample = np.zeros( (step+2,33) )

#=================== 0,1,2,...14
  for r in range(step):
    for c in range(33):
      sample[r,c] = dataset[t+20*r, 2*c]
#===================
  maxindex = -1
  maxlp=0
  for i in range(33):
    ld = dataset[t+predict,2*i]
    if ld > maxlp:
      maxlp = ld
      maxindex = i
  sample[15, maxindex] = 1

#===================
  for i in range(33):
    sample[16,i] = i
  # ArgSort
  lastlap = np.copy(sample[step-1])
  order = lastlap.argsort()
  order = order[::-1]

#===========Regroup
  train_sample = np.zeros( ( step ,40) )
  label = np.zeros(20)

  for i in range(20):
    car_number = order[i]
    for r in range(15):
      train_sample[r , 2*i ] = dataset[t+20*r, 2*car_number]
      train_sample[r , 2*i +1] = dataset[t+20*r, 2*car_number + 1 ]
    label[i] = sample[15,car_number ]

# =======normalization
  lap = math.floor( train_sample[step-1,0])
  for i in range(20):
    train_sample[:, 2*i ] =  train_sample[:, 2*i ]* 0.1 - lap* 0.1 +0.9
#    train_sample[:, 2*i +1] = train_sample[:, 2*i +1]*0.0005
  
  #========random noise
  if noise == True :
    a = random.randint(-10,9)
    a = a*0.1
    b = (random.random()-0.5)*0.02
    for i in range(20):
      train_sample[:, 2*i ] =  train_sample[:, 2*i ] + a
      train_sample[:, 2*i +1] = train_sample[:, 2*i +1] + b
  
  return train_sample, label



The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard
Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [13]:
# Model
n_steps = 15 # 20s *15 step
num_features =40
num_label=20

input_shape = (n_steps, num_features)
model = Sequential()

model.add(LSTM(units=64,
              input_shape=input_shape,
               dropout=0.1,
               return_sequences=True))

model.add(LSTM(units=64,
               dropout=0.1,
               input_shape=input_shape,
               return_sequences=False))

model.add(Dense(64,activation='selu'))
model.add(Dropout(0.1))
model.add(Dense(num_label,activation='softmax'))
#model.add(Dense(2,activation='softmax'))
model.summary()
model.compile(loss='categorical_crossentropy', metrics=['acc','mae','mse'], optimizer=optimizers.Nadam())

#model.save('/content/drive/My Drive/Indycar/LSTM3/model323.h5')

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_4 (LSTM)                (None, 15, 64)            26880     
_________________________________________________________________
lstm_5 (LSTM)                (None, 64)                33024     
_________________________________________________________________
dense_4 (Dense)              (None, 64)                4160      
_________________________________________________________________
dropout_2 (Dropout)          (None, 64)                0         
_________________________________________________________________
dense_5 (Dense)              (None, 20)                1300      
Total params: 65,364
Trainable params: 65,364
Non-trainable params: 0
_________________________________________________________________


In [14]:


# train
n_steps = 15
num_features =40
num_label=20
#num_label=2
sw={0:data0, 1: data1 , 2: data2, 3: data3, 4: data4 , 5: data5}
stest={1:testdata1, 2: testdata2}


pre = 90


test_X = np.empty((0, n_steps, num_features))
test_Y = np.empty((0, num_label))

for n in range(0,1000):
  data = stest[2]

  #t = random.randint(0, data.shape[0]-280 - pre -1 )
  t = n*10 +5
  train_sample, label = create_sample(data , t,pre, False)
  test_X = np.append(test_X, [train_sample], axis=0 )
  test_Y = np.append(test_Y, [label], axis=0 )
  
'''
  if label[0] == 1 :
    test_Y = np.append(test_Y, [np.array([1,0])], axis=0 )
  else:
    test_Y = np.append(test_Y, [np.array([0,1])], axis=0 )
  
'''

naive = np.sum(test_Y[:,0] )/1000
  
print(naive)

for epoch in range(30):
  train_X = np.empty((0, n_steps, num_features))
  train_Y = np.empty((0, num_label))
  samples=2000
  for n in range(0,samples):
    data = sw[random.randint(0,5)]
    t = random.randint(0, data.shape[0]-280 - pre -1)
    train_sample, label = create_sample(data , t,pre, True)
    '''
    if label[0] != 1:
      train_X = np.append(train_X, [train_sample], axis=0 )
      train_Y = np.append(train_Y, [label], axis=0 )
    elif random.randint(0,2)==1 :
      train_X = np.append(train_X, [train_sample], axis=0 )
      train_Y = np.append(train_Y, [label], axis=0 )
    '''
    train_X = np.append(train_X, [train_sample], axis=0 )
    train_Y = np.append(train_Y, [label], axis=0 )
  '''
    if label[0] == 1 :
      train_Y = np.append(train_Y, [np.array([1,0])], axis=0 )
    else:
      train_Y = np.append(train_Y, [np.array([0,1])], axis=0 )
  '''
  model.fit(train_X, train_Y, batch_size = 64 ,epochs=1,  validation_data= (test_X,test_Y ))

#model.predict(test_X,  callbacks=[tensorboard_callback])

#model.save('/content/drive/My Drive/Indycar/LSTM3/model323.h5')

0.793
32/32 [==============================] - 1s 18ms/step - loss: 0.8059 - acc: 0.7590 - mae: 0.0371 - mse: 0.0187 - val_loss: 0.8692 - val_acc: 0.7270 - val_mae: 0.0512 - val_mse: 0.0215
